## Creación de un experimento de interrupción spot

Puede lanzar instancias de spot en capacidad EC2 sobrante con grandes descuentos a cambio de devolverlas cuando Amazon EC2 necesite recuperar la capacidad. Cuando Amazon EC2 reclama una instancia de spot, llamamos a este evento una interrupción de instancia de spot.

Cuando se utilizan instancias de spot, hay que estar preparado para las interrupciones de las instancias de spot. Un método habitual consiste en probar la resistencia de la aplicación simulando interrupciones de subasta. Puede activar las interrupciones de subasta con AWS Fault Injection Simulator (FIS) apuntando a instancias de subasta individuales o a un subconjunto de instancias administradas por servicios como ASG y EC2 Fleet.

Para utilizar AWS FIS, ejecute experimentos en sus recursos de AWS para probar su teoría sobre el desempeño de una aplicación o sistema en condiciones de error. Para ejecutar experimentos, primero debe crear una plantilla de experimento. Una plantilla de experimento es el plano de su experimento. Contiene las acciones, los objetivos y las condiciones de parada del experimento.

In [42]:
import boto3
from botocore.exceptions import ClientError
import json

ec2 = boto3.client('ec2')
ec2_resource = boto3.resource('ec2')
fis = boto3.client('fis')
iam = boto3.client('iam')

region = ec2.meta.region_name
print("Current AWS region:", region)

Current AWS region: us-east-1


### Crear un rol de IAM para AWS FIS

Para utilizar AWS FIS, debe crear un rol de IAM que conceda a AWS FIS los permisos necesarios para que AWS FIS pueda ejecutar experimentos en su nombre. Este rol de experimento se especifica al crear una plantilla de experimento. Para obtener más información, consulte Creación de un rol de IAM para experimentos de AWS FIS.

In [43]:
policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AllowFISExperimentRoleAssumeRole",
            "Effect": "Allow",
            "Principal": {
                "Service": [
                "fis.amazonaws.com"
                ]
            },
            "Action": "sts:AssumeRole"
        }

    ]
}

In [44]:
# Create the role
try:
    response = iam.create_role(
        RoleName='my-fis-role',
        AssumeRolePolicyDocument=json.dumps(policy),
        Description='FIS Experiment Role'
    )
    role_arn = response['Role']['Arn']
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        role_arn = iam.get_role(RoleName='my-fis-role')['Role']['Arn']
        print("Role already exists:", role_arn)
    else:
        print("Unexpected error: %s" % e)

Role already exists: arn:aws:iam::089715336747:role/my-fis-role


El experimento requiere acceso para realizar las acciones ec2:RebootInstances, ec2:StopInstances, ec2:StartInstances, ec2:TerminateInstances , y aws:ec2:send-spot-instance-interruptions en una instancia de EC2. Ejecute el siguiente comando para crear el archivo de configuración fis_role_permissions_policy.json para agregar las políticas de permisos requeridas.

In [45]:
permissions_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AllowFISExperimentRoleEC2Actions",
            "Effect": "Allow",
            "Action": [
                "ec2:RebootInstances",
                "ec2:StopInstances",
                "ec2:StartInstances",
                "ec2:TerminateInstances"
            ],
            "Resource": "arn:aws:ec2:*:*:instance/*"
        },
        {
            "Sid": "AllowFISExperimentRoleSpotInstanceActions",
            "Effect": "Allow",
            "Action": [
                "ec2:SendSpotInstanceInterruptions"
            ],
            "Resource": "arn:aws:ec2:*:*:instance/*"
        }
    ]
}

In [46]:
# Put the policy to the role created earlier
response = iam.put_role_policy(
    RoleName='my-fis-role',
    PolicyName='my-fis-policy',
    PolicyDocument=json.dumps(permissions_policy)
)


### Cree la plantilla del experimento de interrupción del spot

En este paso se crea una plantilla de experimento que envía la interrupción de subasta a las instancias de subasta lanzadas a través del grupo EC2 Auto Scaling o la flota EC2 en capítulos anteriores. La plantilla de experimento de AWS FIS contiene:

* **Una acción**, es una actividad que AWS FIS realiza en un recurso de AWS durante un experimento. En este caso, se utiliza una acción preconfigurada aws:ec2:send-spot-instance-interruptions, que envía un aviso de interrupción de instancia de spot a las instancias de spot objetivo dos minutos antes de interrumpirlas. La acción también envía una recomendación de reequilibrio de instancias EC2 basada en el parámetro durationBeforeInterruption. En este caso se establece la acción preconfigurada aws:ec2:send-spot-instance-interruptions con durationBeforeInterruption establecida en 2 minutos. Puede cambiar a un valor superior a 2 minutos para representar una recomendación de reequilibrio de instancia EC2 que se adelante al aviso de interrupción de instancia spot.

* **Un objetivo**, es uno o más recursos de AWS en los que el experimento de AWS FIS realiza una acción durante un experimento. En este caso, se establece un objetivo con aws:ec2:spot-instance como tipo de recurso y se filtran las instancias de spot lanzadas a través de ASG utilizando resourceTags establecidas en "aws:autoscaling:groupName": "EC2SpotWorkshopASG".

* **Una condición de parada**, es un mecanismo de AWS FIS para detener un experimento si alcanza un umbral que se define como una alarma de Amazon CloudWatch. En este caso, el experimento se ejecuta sin condición de parada.

In [47]:
actions = {
    "interruptSpotInstance": {
        "actionId": "aws:ec2:send-spot-instance-interruptions",
        "parameters": {
            "durationBeforeInterruption": "PT2M"
        },
        "targets": {
            "SpotInstances": "SpotInstancesInASG"
        }
    }
}
targets = {
    "SpotInstancesInASG": {
        "resourceType": "aws:ec2:spot-instance",
        "resourceTags": {
            "aws:autoscaling:groupName": "EC2SpotWorkshopASG"
        },
        "filters": [
            {
                "path": "State.Name",
                "values": [
                    "running"
                ]
            }
        ],
        "selectionMode": "PERCENT(50)"
    }
}

In [48]:
# Create the experiment template
response = fis.create_experiment_template(
    actions=actions,
    targets=targets,
    stopConditions=[
        {
            'source': 'none'
        },
    ],
    roleArn=role_arn,
    description="Test Spot Instance interruptions")

fis_template_id = response['experimentTemplate']['id']

### Ejecutar el experimento Interrupción del spot

Para ejecutar el experimento, utilice el comando start-experiment para ejecutar la plantilla creada anteriormente. El experimento puede ejecutarse varias veces para validar los resultados de la ejecución de su aplicación en EC2 Spot.

In [49]:
response = fis.start_experiment(experimentTemplateId=fis_template_id)
if response['ResponseMetadata']['HTTPStatusCode'] == 200:   
    print("Experiment started successfully")

Experiment started successfully


Como resultado de este experimento, se observa que el 50% de las instancias de spot lanzadas por el grupo Auto Scaling reciben las señales de recomendación de reequilibrio. Tenga en cuenta que esto es inferior a la Cuota de servicio de 5, que es el número máximo de instancia de spot EC2 que puede interrumpir un solo experimento. Cuando se completan las acciones de este experimento

* La instancia de spot de destino recibe una señal de recomendación de reequilibrio de instancias.
* Se emite un aviso de interrupción de instancia de spot dos minutos antes de que Amazon EC2 termine su instancia.
* Transcurridos dos minutos, se termina la instancia de spot.